### Preprocessing

First, we normalize all the MRIs because pixels values are expressed in Houndsfield Units. And since samples were generated by different MRI scanners, their range are not directly comparable.

In [ ]:
for i in range(n_train_samples):
    for j in range(0,4,2):
        for k in range(mri_train[i][j].shape[0]):
            mri_train[i][j][k]=normalize(mri_train[i][j][k])
    
for i in range(n_test_samples):
    for j in range(0,4,2):
        for k in range(mri_test[i][j].shape[0]):
            mri_test[i][j][k]=normalize(mri_test[i][j][k])

In [ ]:
patient_idx=1
ed_mri_idx=0
ed_mask_idx=1

plot_slices(mri_train[patient_idx][ed_mri_idx], title=f'ED MRI of patient {patient_idx}')
plot_slices(mri_train[patient_idx][ed_mask_idx], title=f'ED segmentation of patient {patient_idx}')

In [ ]:
from skimage.filters import gaussian

mri=mri_test[patient_idx][ed_mri_idx]

sigma_gauss=1.5
size_gauss=7 #49 coefficients
gauss_filter=lambda x: gaussian(x, sigma=sigma_gauss, truncate=size_gauss/2)

mri_gauss=np.vectorize(gauss_filter, signature='(n,m)->(n,m)')(mri)
# plot_slices(slices=mri_gauss, title='Gaussian filtered MRI')
plot_slices(slices=mri_gauss, title='Gaussian filtered MRI')

Then we use an edge detector by applying a Canny filter. (as recommended by the article)

In [ ]:
from skimage.feature import canny

sigma_canny=5
canny_filter=lambda x: canny(x, sigma=sigma_canny)

mri_canny=np.vectorize(canny_filter, signature='(n,m)->(n,m)')(mri_gauss)
plot_slices(mri_canny, title='Canny filtered MRI')

Next we compute Gradient Vector Flow Forces using GVF algorithm

In [ ]:
from skimage.filters import sobel_h, sobel_v, laplace

def GVF(edgemap, mu=0.02, iterations=100):
    """Gradient Vector Flow algorithm for edge detection"""
    # Initialize
    u = np.zeros(edgemap.shape)
    v = np.zeros(edgemap.shape)
    u[1:-1, 1:-1] = edgemap[1:-1, 1:-1]
    v[1:-1, 1:-1] = edgemap[1:-1, 1:-1]

    for i in range(iterations):
        # We use laplacian
        u[1:-1, 1:-1] = (u[1:-1, 1:-1] + mu * (u[2:, 1:-1] + u[:-2, 1:-1] + u[1:-1, 2:] + u[1:-1, :-2] - 4 * u[1:-1, 1:-1])) / (1 + 4 * mu)
        v[1:-1, 1:-1] = (v[1:-1, 1:-1] + mu * (v[2:, 1:-1] + v[:-2, 1:-1] + v[1:-1, 2:] + v[1:-1, :-2] - 4 * v[1:-1, 1:-1])) / (1 + 4 * mu)

    return u, v

def GVF_filter(edgemap, mu=0.02, iterations=50):
    """Filter an edge map using GVF"""
    u, v = GVF(edgemap, mu=mu, iterations=iterations)
    return np.sqrt(u**2 + v**2)

mri_gvf=np.vectorize(GVF_filter, signature='(n,m)->(n,m)')(mri_canny)
plot_slices(mri_gvf, title='GVF filtered MRI')


In [ ]:
from skimage.segmentation import active_contour

slc=mri_gvf[8].copy()
slc=(slc-slc.min())/(slc.max()-slc.min())

c_x,c_y=slc.shape[0]//2, slc.shape[1]//2
r0=50
s=np.linspace(0,2*np.pi,25)
r=c_x+r0*np.sin(s)
c=c_y+r0*np.cos(s)
init=np.array([r,c]).T
n_iter=100

snake=active_contour(slc, init, alpha=0.01, beta=0.5, gamma=0.5, convergence=1e-5)

plt.imshow(slc)
plt.plot(init[:,1], init[:,0], '-r', lw=3)
plt.plot(snake[:,1], snake[:,0], '-g', lw=4)
plt.show()

## CODE FOR HIST EQUALIZATION

In [ ]:
from skimage.exposure import equalize_adapthist

slice=mri_train[2][0][1]

plt.figure()
plt.imshow(equalize_adapthist(slice/255))


plt.figure()
plt.imshow(slice)